In [5]:
%load_ext autoreload
from quantifier import *
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload 2
lidf=readin()

In [11]:
lidf

,Lane,Sample,Volume,MW,wcount
0,1,std,5,NaN,NaN
1,2,std,15,NaN,NaN
2,4,E3,3,78.571934,21.0
3,5,E5,3,27.631576,7.0
4,6,E7,3,39.429542,12.0
5,7,E8,3,35.309518,7.0
6,8,F1,3,56.512896,12.0
7,9,F2,3,73.673092,7.0
8,10,F3,3,64.820306,18.0
9,11,F6,3,66.645218,22.0
